## Rohlik Sales Forecasting Challenge

**This example demonstrates pytorch-graphstam library's usage for a typical, non-hierarchical multistep forecasting problem**

- The solo model achieves a score of ~18.33 i.e. ~top-20 on private leaderboard
- An ensemble of this model with xgboost achieves a score of ~17.3 i.e. 4th place on private leaderboard

**Competition Overview**  
The Rohlik Sales Forecasting Challenge on Kaggle is centered around predicting daily sales for a leading European online grocery retailer, Rohlik. Participants are given historical sales data for numerous products and are tasked with forecasting future sales. 

**Competition Links**  
- [Rohlik Sales Forecasting Challenge (v2) Main Page](https://www.kaggle.com/competitions/rohlik-sales-forecasting-challenge-v2/overview)  
- [Data Overview & Download](https://www.kaggle.com/competitions/rohlik-sales-forecasting-challenge-v2/data)  
- [Evaluation Details](https://www.kaggle.com/competitions/rohlik-sales-forecasting-challenge-v2/overview/evaluation)  

---


### Prerequisites

#### install pytorch >= v2.5.x with CUDA version 12.1, also works with with cu118/cu124/cu128 
pip install torch==2.5.1 --index-url https://download.pytorch.org/whl/cu121

#### install pytorch-geometric >= v2.6.x
check version compatibility with torch & os here: https://pytorch-geometric.readthedocs.io/en/latest/notes/installation.html
pip install torch_geometric

#### install pytorch-geometric dependencies for torch v2.5.* & CUDA v12.1
pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.5.0+cu121.html
    
#### install pytorch-graphstam
pip install -U git+https://github.com/rsscml/pytorch-graphstam


In [ ]:
# core library imports

from pytorch_graphstam import graphstam
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings("ignore")

pd.set_option("display.max_columns", 200)
pd.set_option("display.max_rows", 100)

### Data Preparation

In [ ]:
# read in the datafiles -- assuming they are in the current directory
train_sales_df = pd.read_csv("sales_train.csv", parse_dates=['date'])
test_sales_df = pd.read_csv("sales_test.csv", parse_dates=['date'])
inventory_df = pd.read_csv("inventory.csv")
calendar_df = pd.read_csv("calendar.csv", parse_dates=['date'])
weights_df = pd.read_csv("test_weights.csv")

# merge the above datasets into a single dataset
# add 'availability' col to test_Sales & set it to null

test_sales_df['availability'] = np.nan
sales_df = pd.concat([train_sales_df, test_sales_df], axis=0)
sales_df = sales_df.sort_values(by=['unique_id','date'], ascending=True).reset_index(drop=True)
# merge inventory
sales_df = sales_df.merge(inventory_df, on=['unique_id','warehouse'], how='left')
# merge calendar
calendar_df = pd.get_dummies(calendar_df, columns=['holiday_name'])
sales_df = sales_df.merge(calendar_df, on=['date','warehouse'], how='left')
# merge weights
sales_df = sales_df.merge(weights_df, on=['unique_id'], how='left')

# create date features
sales_df['weekday'] = sales_df['date'].dt.weekday
sales_df['week'] = sales_df['date'].dt.isocalendar().week
sales_df['month'] = sales_df['date'].dt.month
sales_df['year'] = sales_df['date'].dt.year
sales_df['prev_year'] = (sales_df['date'] - timedelta(days=1)).dt.year
sales_df['day'] = sales_df['date'].dt.day
sales_df['is_month_start'] = sales_df['date'].dt.is_month_start
sales_df['is_month_end'] = sales_df['date'].dt.is_month_end
sales_df['quarter'] = sales_df['date'].dt.quarter
sales_df['weekend']=(sales_df['weekday']>4).astype(np.int8)
sales_df['days_since_2020'] = (sales_df['date'] - pd.to_datetime('2020-01-01')).dt.days.astype('int')

# add 'country' as a feature
store2country = {
        'Budapest_1': 'Hungary',
        'Prague_2': 'Czechia',
        'Brno_1': 'Czechia',
        'Prague_1': 'Czechia',
        'Prague_3': 'Czechia',
        'Munich_1': 'Germany',
        'Frankfurt_1': 'Germany'
}

sales_df['country'] = sales_df['warehouse'].apply(lambda x:store2country[x])

# Basic fillna ['total_orders','sales','availability'] as mean of date
sales_df['sales'] = sales_df['sales'].fillna(sales_df.groupby(['unique_id','weekday'])['sales'].transform('mean'))
sales_df['total_orders'] = sales_df['total_orders'].fillna(sales_df.groupby(['unique_id','weekday'])['total_orders'].transform('mean'))
sales_df['availability'] = sales_df['availability'].fillna(sales_df.groupby(['unique_id','weekday'])['availability'].transform('mean'))
# fill 0 for any edge cases
sales_df['sales'] = sales_df['sales'].fillna(0)
sales_df['availability'] = sales_df['availability'].fillna(0)

# check for nulls in any columns
print(sales_df.columns[sales_df.isnull().any()])

# writeout the processed dataset
sales_df.to_csv("sales_processed.csv", index=False)

### Main

In [ ]:
# read in the processed dataset as a pandas dataframe
df = pd.read_csv("sales_processed.csv", dtype={'unique_id':'str', 'date':'str'})

# optional step: restrict dataset to >= 2021 as prior data is patchy
df = df[df['year']>=2021]

# optional step: power transform target col using square root
df['sales'] = np.sqrt(df['sales'])

# the submission requires forecasts for only a subset of unique_ids; extract these ids for re-weighting purpose later
solution_df = pd.read_csv("solution.csv")
solution_df['unique_id'] = solution_df['id'].str.split('_').str[0]
all_solution_unique_ids = solution_df['unique_id'].unique().tolist()

print(len(all_solution_unique_ids)) # show show 3625

In [ ]:
# create edge basis columns i.e., these columns can be used to connect product nodes through edges on the basis of some similarity

df['common_name'] = df['name'].str.split('_').str[0]
df['warehouse_common_name'] = df['warehouse'] + '_' + df['common_name']
df['warehouse_name'] = df['warehouse'] + '_' + df['name']
df['warehouse_L3_category_name_en'] = df['warehouse'] + '_' + df['L3_category_name_en']
df['warehouse_L4_category_name_en'] = df['warehouse'] + '_' + df['L4_category_name_en']
df['warehouse_L2_category_name_en'] = df['warehouse'] + '_' + df['L2_category_name_en']
df['warehouse_L1_category_name_en'] = df['warehouse'] + '_' + df['L1_category_name_en']
df['warehouse_L4_L3_category_name_en'] = df['warehouse'] + '_' + df['L4_category_name_en']+ '_' + df['L3_category_name_en']


In [ ]:
# identify all the columns in the dataset we plan to use 

id_col = 'unique_id'
datetime_col = 'date'
target_col = 'sales'
weight_col = 'weight'
date_features = ['weekday','week','month','day','is_month_start','is_month_end','weekend']
other_features = ['total_orders', 'sell_price_main']
promo_features = ['type_0_discount',
                  'type_1_discount',
                  'type_2_discount',
                  'type_3_discount',
                  'type_4_discount',
                  'type_5_discount',
                  'type_6_discount']


event_features = ['holiday',
                 'shops_closed', 
                 'winter_school_holidays',
                 'school_holidays',
                 'holiday_name_1st Christmas Day',
                 'holiday_name_2nd Christmas Day',
                 'holiday_name_All Saints Day',
                 "holiday_name_All Saints' Day Holiday",
                 'holiday_name_Ascension day',
                 'holiday_name_Assumption of the Virgin Mary',
                 'holiday_name_Christmas Eve',
                 'holiday_name_Christmas Holiday',
                 'holiday_name_Corpus Christi',
                 'holiday_name_Cyrila a Metodej',
                 'holiday_name_Day of National Unity',
                 'holiday_name_Den boje za svobodu a demokracii',
                 'holiday_name_Den ceske statnosti',
                 'holiday_name_Den osvobozeni',
                 'holiday_name_Den vzniku samostatneho ceskoslovenskeho statu',
                 'holiday_name_Easter Monday',
                 'holiday_name_Epiphany',
                 'holiday_name_German Unity Day',
                 'holiday_name_Good Friday',
                 'holiday_name_Hungary National Day Holiday',
                 'holiday_name_Independent Hungary Day',
                 'holiday_name_Jan Hus',
                 'holiday_name_Labour Day',
                 'holiday_name_Memorial Day of the Republic',
                 'holiday_name_Memorial day of the 1956 Revolution',
                 'holiday_name_National Defense Day',
                 'holiday_name_New Years Day',
                 'holiday_name_Reformation Day',
                 'holiday_name_State Foundation Day',
                 'holiday_name_Whit monday',
                 'holiday_name_Whit sunday']
                  
global_context_cols = ['country',
                       'warehouse',
                       'name',
                       'L1_category_name_en',
                       'L2_category_name_en', 
                       'L3_category_name_en',
                       'L4_category_name_en']

static_context_cols = ['warehouse_L2_category_name_en', 'warehouse_L3_category_name_en','warehouse_common_name']


#### A bit about global_context_cols & static_context_cols

- global_context_cols: This is a list of static, categorical columns which are to be used as 'context' for a forecast key. The context is provided to a key node through an in-coming directed edge. In the present case, we want every key node to have a context comprising of the country & warehouse to which it belongs, various L1/L2/L3 & L4 categories under which it comes & the product name. 

- static_context_cols: This is a list of static, categorical columns which are to be used as the basis for drawing point to point bidirectional edges between key nodes. In the present case, we want to connect all the key nodes that are common to a warehouse & the L2 category in that warehouse, so, the basis for inferring such edges is the column: 'warehouse_L2_category_name_en'. Similarily, we connect all the key nodes common to a warehouse & the L3 category using 'warehouse_L3_category_name_en' & all the key nodes common to a warehouse & having a common product name are connected using 'warehouse_common_name'.

Note 1: The two lists are mutually exclusive, it may require some trial to figure out which columns should be in which list.

Note 2: Since columns in the 'static_context_cols' list form the basis for p2p edges between the key nodes, it has the biggest impact on size of the graph & speed of computation. 

In [ ]:
# this is a helper function used to determine the non-default weights for various forecast_keys

def create_weights(df):
    max_test_wt = df[df['unique_id'].isin(all_solution_unique_ids)]['weight'].unique().max()
    q25_test_wt = np.quantile(df['weight'].unique(), q=0.25)
    df['weight'] = np.clip(df['weight'], a_min=q25_test_wt, a_max=max_test_wt)
    return df


#### Define config

In [ ]:
# train/test cutoffs
train_till = "2024-04-30"
test_till = "2024-06-02"

# infer cutoffs
infer_start = "2024-06-03"
infer_end = "2024-06-16"


The following config is meant to be used for the **GraphTFT** model.

To view complete config details, run the following:
- graphstam.show_config_template()

In [ ]:
# build config for GraphTFT model


# base features
features_config = {
                    "id_col": id_col,
                    "target_col": target_col,      
                    "time_index_col": datetime_col,      
                    "global_context_col_list": global_context_cols,   
                    "static_cat_col_list": static_context_cols,       
                    "temporal_known_num_col_list": event_features + other_features + date_features + promo_features,  
                    "temporal_unknown_num_col_list": [],  
                    "wt_col": weight_col
}

# Optional: 
# rolling features: use moving averages & std.dev over various window sizes of 7,14 & 28 days.

rolling_features =  [(datetime_col,'mean', 7, 0),
                     (datetime_col,'mean', 14, 0),
                     (datetime_col,'std', 14, 0),
                     (datetime_col,'mean', 28, 0),
                     (datetime_col,'std', 28, 0)]

# data config
data_config = {
                "max_lags": 60,        # max lookback period
                "fh": 14,              # forecast horizon
                "train_till": train_till, 
                "test_till": test_till,
                "scaling_method": 'mean_scaling', # scales target using the mean scaling method
                "tweedie_out": False,             # useful in cases where the target variable is tweedie or poisson distributed  
                "tweedie_variance_power": [1.1],  # the hyperparameter for tweedie distribution
                "interleave": 1,                  # if == 1, trains on all samples in the dataset, if 2, trains on half the samples & so on 
                "recency_weights": False,         # gives greater importance more recent samples
                "recency_alpha": 1,               # controls the skew of recency_weights, greater this value more heavily the recent samples are weighted
}


# model config
model_config = {
                "model_dim": 64,                  # defines model dimensionality & determines the no. of parameters in the model
                "num_gnn_layers": 1,              # no. of gnn layers to use (in the encoder side of TFT) 
                "num_attn_layers": 4,             # no. of attention layers to use. 2-4 should be sufficient for most cases. 
                "num_rnn_layers": 1,              # the no. of rnn layers to use (this is again a component of the TFT architecture)
                "heads": 1,                       # no. of heads for attention 
                "dropout": 0,                     # dropout regularization
                "device": 'cuda',                 # 'cuda': run on GPU, else, 'cpu' -- not practical!
                "gbt": False,                     # if True, fits a booster model to the residuals of the base model i.e., trains > 1 model (given by n_boosters)
                "n_boosters": 1,                  # if gbt == True, no. of booster models to use; > 1 is impractical in most cases
}


# train config
# Scheduler params - default works well - for more information check pytorch documentation

scheduler_params_config = {
                            "factor": 0.5, 
                            "patience": 5, 
                            "threshold": 0.0001, 
                            "min_lr": 0.0001, 
                            "clip_gradients": False, 
                            "max_norm": 2.0, 
                            "norm_type": 2
} 


train_config = {
                "lr": 0.001,                       # learning rate  
                "min_epochs": 10,                  # minimum passes over the training dataset
                "max_epochs": 100,                 # max. epochs
                "patience": 5,                     # no. of epochs to wait for improvement in loss   
                "model_prefix": "rohlik",          # used to determine the name of the saved models
                "loss_type": 'Huber',              # loss fn. other options are: ['RMSE','Quantile','Tweedie']
                "huber_delta": 0.5,                # hyperparam for the huber loss fn.
                "use_amp": True,                   # autocast float32 to float16 to save memory at the expense of accuracy  
                "use_lr_scheduler": True,          # whether to use decreasing learning rate schedule 
                "scheduler_params": scheduler_params_config, # params for the lr scehduler
                "sample_weights": True             # whether to use sample weighted loss for minimization  
}    


# infer config
infer_config = {
                "infer_start": infer_start,
                "infer_end": infer_end,
}


# final config
config = {
            "model_type": 'GraphTFT',              # other options: ['GraphSeq2Seq', 'SimpleGraph']
            "working_dir": './lib_tft_dir',        # Save models & logs here
            "features_config": features_config,
            "rolling_features": rolling_features,
            "data_config": data_config,
            "model_config": model_config,
            "train_config": train_config,
            "infer_config": infer_config
}
   

#### Train & Infer

In [ ]:
# train: one model per country

for country in df['country'].unique().tolist():
    print("\ntraining model for country: ", country)
    df_country = df[df['country']==country]
    df_country = create_weights(df_country)
    print("no. of forecast keys: ", df_country['unique_id'].nunique())
    
    # create main 'gml' object
    gmlobj = graphstam.gml(config)
    
    # build model & initialize
    gmlobj.build(df_country)
    
    # train
    gmlobj.train()
    
    # infer - saves output as csv at: forecast_filepath 
    forecast_df = gmlobj.infer(forecast_filepath=f"./output/rohlik_tft_{country}.csv")


#### Post-process for submission -- optional step

In [ ]:
# merge country forecast files
df_hungary = pd.read_csv("./output/rohlik_tft_Hungary.csv")
df_germany = pd.read_csv("./output/rohlik_tft_Germany.csv")
df_czechia = pd.read_csv("./output/rohlik_tft_Czechia.csv")

out_df = pd.concat([df_hungary, df_germany, df_czechia], axis=0)

# merge with main df
out_df = out_df.merge(sales_df, on=['name','warehouse'], how='left')

# format as required
out_df['id'] = out_df['unique_id_'].astype(str) + '_' + out_df['date'].astype(str)
out_df = out_df[['id','forecast']]

# merge with submission file
submit_df = pd.read_csv("solution.csv")
submit_df = submit_df.merge(out_df, on=['id'], how='inner')
submit_df = submit_df.drop(columns=['sales_hat']).rename(columns={'forecast':'sales_hat'})
# clip min. at 0
submit_df['sales_hat'] = np.where(submit_df['sales_hat']<0, 0, submit_df['sales_hat'])
# undo power transformation
submit_df['sales_hat'] = np.power(submit_df['sales_hat'], 2)

# final submission file
submit_df.to_csv("sample_submission_gtft.csv", index=False)
